<section class="section1"><h1>Lab week 8</h1>
<p>We have seen control flow, loops, and <code>numpy</code> arrays. Now we are going to use them on the simplex method.</p>
<section class="section2"><h2>Script</h2>
<p>Linear programming is a core part of operational research. We'll look at solving the following linear program
<span>$
\begin{array}{cccc}
  \max &amp; x_1    &amp; + x_2  \\
       &amp; 2 x_1  &amp; - x_2 &amp; \le 4 \\
       &amp; x_1    &amp; + 2 x_2 &amp; \le 3
\end{array}
$</span>
where <span>$x_1, x_2 \ge 0$</span>. We'll code up the simplex method.</p>
<p>Let us bring the problem to standard form by introducing slack variables <span>$s_1, s_2 \ge 0$</span> and transforming the problem into a minimization problem:</p>
<span>$
\begin{array}{cccccc}
  \min &amp; -x_1   &amp; -x_2  \\
       &amp; 2 x_1  &amp; - x_2   &amp; + s_1 &amp;       &amp; = 4 \\
       &amp; x_1    &amp; + 2 x_2 &amp;       &amp; + s_2 &amp; = 3
\end{array}
$</span>
<p>The corresponding tableau reads:
<span>$
\left(\begin{array}{c|cccc}
0 &amp; -1 &amp; -1 &amp; 0 &amp; 0 \\
\hline
4 &amp;  2 &amp; -1 &amp; 1 &amp; 0 \\
3 &amp;  1 &amp;  2 &amp; 0 &amp; 1
\end{array}
\right)
$</span>
</p>
<p>Recall that we count rows and columns from zero here.</p>
<p>The tableau contains the reduced cost coefficients (row <span>$0$</span>), the right-hand side (RHS) coefficients (column <span>$0$</span>, ignoring its first component), which should be kept nonnegative at all times, and the coefficients of the various constraints. To be basic feasible, a collection of its columns should form an identity matrix (which is the case here: consider columns <span>$3,4$</span>).</p>
<p>From an algebraic perspective (slightly more abstract than the one adopted in the lectures), the simplex method performs a sequence of row operations (pivoting operations) to remove all negative numbers from the top row (row <span>$0$</span>), at each stage choosing a pivot which guarantees that the nonnegativity of the RHS coefficients (column <span>$0$</span>, ignoring is first entry, i.e., the entry in position <span>$(0, 0)$</span>) is preserved.</p>
<p>First, we want to store the tableau. We'll do this using <code>numpy.</code></p>
</section></section>

In [ ]:
import numpy

In [ ]:
tableau = numpy.array([ [0, -1, -1, 0, 0],
                        [4,  2,  -1, 1, 0],
                        [3,  1,  2, 0, 1] ], dtype=numpy.float64)
print(tableau)

In [ ]:
[[ 0. -1. -1.  0.  0.]
 [ 4.  2. -1.  1.  0.]
 [ 3.  1.  2.  0.  1.]]


<p>We've explicitly said that the tableau entries are floating point numbers here, as we know we'll end up with non-integer results.</p>
<p>Note that the vector of right-hand sides has been stored in column <code>0</code>, from row <code>1</code> to row <code>2</code>. The entry in position <code>(0,0)</code> corresponds to the opposite of the objective function value.</p>
<p>First, we inspect the row of reduced costs (row <code>0</code>).</p>

In [ ]:
print(tableau[0, :])

In [ ]:
[ 0. -1. -1.  0.  0.]


<p>We see negative entries in columns <code>1</code> and <code>2</code>. Applying Bland's rule, we pivot on the left-most column: column <code>1</code>. As, carrying out the min ratio test, we observe <code>4/2 &lt; 3/1</code>, we select row <code>1</code>, pivoting in position <code>(1,1)</code>.</p>
<p>First, we divide row 1 through by 2 (the entry in the pivot position):</p>

In [ ]:
tableau[1, :] = tableau[1, :] / tableau[1,1]
print(tableau)

In [ ]:
[[ 0.  -1.  -1.   0.   0. ]
 [ 2.   1.  -0.5  0.5  0. ]
 [ 3.   1.   2.   0.   1. ]]


<p>Next, we subtract the new row <code>1</code> scaled by <code>-1</code> (the entry in position <code>(0,1)</code>) from row <code>0</code>:</p>

In [ ]:
tableau[0, :] = tableau[0, :] - tableau[0, 1] * tableau[1, :]
print(tableau)

In [ ]:
[[ 2.   0.  -1.5  0.5  0. ]
 [ 2.   1.  -0.5  0.5  0. ]
 [ 3.   1.   2.   0.   1. ]]


<p>Next, we subtract the new row <code>1</code> scaled by <code>1</code> (the entry in position <code>(2,1)</code>) from row <code>2</code>:</p>

In [ ]:
tableau[2, :] = tableau[2, :] - tableau[2, 1] * tableau[1, :]
print(tableau)

In [ ]:
[[ 2.   0.  -1.5  0.5  0. ]
 [ 2.   1.  -0.5  0.5  0. ]
 [ 1.   0.   2.5 -0.5  1. ]]


<p>After this, a column of the identity matrix shows up in colum <code>1</code>.
Row <code>0</code> (the reduced costs row) still contains a negative value in column <code>2</code>. As the column contains a single positive coefficient, the minumum ratio test is not necessary. Pivoting takes place in position <code>(2,2)</code>.
First, we divide row <code>2</code> by <code>2.5</code>:</p>

In [ ]:
tableau[2, :] = tableau[2, :] / tableau[2, 2]
print(tableau)

In [ ]:
[[ 2.   0.  -1.5  0.5  0. ]
 [ 2.   1.  -0.5  0.5  0. ]
 [ 0.4  0.   1.  -0.2  0.4]]


<p>Then, we subtract the new row <code>2</code> from rows <code>0</code> and <code>1</code>, scaled by, respectively, <code>-1.5</code> and <code>-0.5</code>. We obtain:</p>

In [ ]:
tableau[0, :] = tableau[0, :] - tableau[0, 2] * tableau[2, :]
tableau[1, :] = tableau[1, :] - tableau[1, 2] * tableau[2, :]
print(tableau)

In [ ]:
[[ 2.6  0.   0.   0.2  0.6]
 [ 2.2  1.   0.   0.4  0.2]
 [ 0.4  0.   1.  -0.2  0.4]]


<p>Since the reduce cost vector (row <code>0</code>) is now componentwise nonnegative, we have found an optimal solution. Note that, as the two columns of the identity matrix are in columns <code>1</code> (corresponding to <span>$x_1$</span>) and <code>2</code> (corresponding to <span>$x_2$</span>), we deduce <span>$x_B = (x_1, x_2)$</span>. We can read the value of both variables and the corresponding objective function value as follows (remember to flip the sign of the element in position <code>(0,0)</code> as the value in <code>(0,0)</code> is the opposite of the objective function value):</p>

In [ ]:
print("z =", -tableau[0, 0])
print("x_1 =", tableau[1, 0])
print("x_2 =", tableau[2, 0])

In [ ]:
z = -2.6
x_1 = 2.2
x_2 = 0.4


<section class="section3"><h3>Working with parts of the tableau</h3>
<p>To turn the steps above into a <em>general</em> function we need to find the candidate column on which we can pivot, and then the best row (out of the set of candidate rows) on which to actually pivot. This means getting a list, or array, containing the row and/or column indexes that we want to use.</p>
<p>With <code>numpy</code> directly we can check whether something is true or false using standard logical comparison operations. For example, let us rebuild our <code>tableau</code> and check for negative cost coefficients:</p>
</section>

In [ ]:
tableau = numpy.array([ [0, -1, -1, 0, 0],
                        [4,  2,  -1, 1, 0],
                        [3,  1,  2, 0, 1] ], dtype=numpy.float64)
print(tableau[0, 1:] < 0)

In [ ]:
[ True  True False False]


<p>We excluded the first column, as that is the objective function, not a cost coefficient.</p>
<p>To get the indexes, we can then use the <code>nonzero</code> function (as <code>False</code> is equivalent to <code>0</code>, this tells us the indexes where the comparison is <code>True</code>):</p>

In [ ]:
negative_cost_idxs = numpy.nonzero(tableau[0, 1:] < 0)[0]
print(negative_cost_idxs)

In [ ]:
[0 1]


<p><code>nonzero</code> returns a tuple, giving us the index in each dimension of the array. As we're dealing with a 1d array we only care about the first dimension, hence the <code>[0]</code> at the end.</p>
<p>Note that we could (should!) check that any costs are negative. If not, then we have reached optimality. We can do this by using <code>len</code> to check the number of appropriate columns.</p>
<p>This tells us that the first and second cost coefficients are negative, and therefore are candidate columns. We can therefore choose the first of these as the column on which to pivot:</p>

In [ ]:
column = negative_cost_idxs[0] + 1

<p>We had to add <code>1</code> in order to get the index into the <code>tableau</code>, as we had ignored the first column (as it corresponds to the objective function).</p>
<p>We can perform similar operations to find the row on which to pivot. First, store the row indexes where the entry is positive (as this is a requirement for Bland's algorithm):</p>

In [ ]:
positive_tableau_idxs = numpy.nonzero(tableau[1:, column] > 0)[0]
print(positive_tableau_idxs)

In [ ]:
[0 1]


<p>We can now use <code>argmin</code> to find the index with the minimum ratio, in order to do the minimum ratio test:</p>

In [ ]:
ratio = tableau[1:, 0] / tableau[1:, column]  # Compute all ratios
row_argmin = numpy.argmin(ratio[positive_tableau_idxs])  # Only check positive entries
row = positive_tableau_idxs[row_argmin] + 1 # Correct for ignoring first row
print(row)

In [ ]:
1


<section class="section5"><h5>Exercise</h5>
<p>Turn the operations we carried out above into a function <code>simplex_method</code> which takes a basic feasible tableau as input and returns 1) the status (as a string) of the problem (either optimal or unbounded) and, for bounded problems, 2) the optimal solution value and 3) an optimal solution. It's up to you to decide what to return for unbounded problems. The method should apply Bland's rule when choosing what the pivoting pair of row and column.</p>
<p>Remember that, given a <code>numpy</code> array <code>tableau</code>, <code>tableau.shape</code> is a list of the sizes of each dimension.</p>
</section><section class="section5"><h5>Exercise</h5>
<p>Check the correctness of your function by running on the problems seen in the lectures and problem classes/sheets.</p>
</section>